# Chapter 19: Interacting with Tkinter GUIs

Here, we'll look at accepting user input in a GUI.

To reiterate, a Tkinter GUI is a main window that contains a number of widgets.

When you call mainloop on the window, it starts listening for input and dispatching events to the widgets as the user interacts with them. So, in order to have a functional program, each widget needs to know how to respond to input.

In [ ]:
from tkinter import Tk, Frame, Button

def chattyButton():
    #Here, I'll make a window that contains a button. When the user
    #clicks on the button, it will print a message.
    window = Tk()
    buttonHolder = Frame(window, padx=10, pady=10, background="black")
    #Now, before I make the button, I'll create a function that describes what
    #should happen when the button is pressed. This function will be called
    #by the button when the user clicks on it. 
    def sayMessage():
        print("Button clicked!")
    theButton = Button(buttonHolder, text="Say something", command=sayMessage)
    theButton.pack()
    buttonHolder.pack()
    window.mainloop()


#This goes back to some of the topics discussed in chapter 13 involving
#closures and scope. Consider the following code:

def closureButton():
    window=Tk()
    #For the reason that this must be a list, see chapter 13. sayMessage can't
    #reassign a variable in its containing scope, but it can modify what that
    #variable points to. If timesClicked was just the number 1, you'd be trying
    #to modify the number 1 itself.
    #Python provides the very unhelpful message "local variable referenced
    #before assignment" if you try to do this, which really doesn't explain
    #the problem. 
    timesClicked=[0]
    def sayMessage():
        print("Button clicked {0:d} times".format(timesClicked[0]))
        timesClicked[0]+= 1
    theButton = Button(window, text="say times", command=sayMessage)
    theButton.pack()
    window.mainloop()

Great. Let's play with this concept by writing a calculator.

A, uh, very simple calculator.

It supports addition, subtraction, multiplication, and the numbers one to nine. It's stack-based, which is kind of odd if you've not seen it. Basically, there's a stack of numbers.
When you enter a number it's pushed onto the stack. Then, operations remove the top two numbers and push the result onto the stack.

So let's trace the behavior of the calculator

In [ ]:
#INPUT      STACK
#2          {2]
#3          [2, 3]
#8          [2, 3, 8]
#+          [2, 11]
#4          [2, 11, 4]
#*          [2, 44]
#-          [-42]

See how the operators work on the last two values? Good. It's a lot easier to program a calculator this way, so that's why we're doing it.

Let's implement the logic of the calculator in a function:

In [ ]:
def stackOperation(op, stack):
    if(op == "+"):
        stack = stack[:-2] + [stack[-2] + stack[-1]]
    elif(op == "-"):
        stack = stack[:-2] + [stack[-2] - stack[-1]]
    elif(op == "*"):
        stack = stack[:-2] + [stack[-2] * stack[-1]]
    else: 
        stack = stack + [op]
    return stack

#Now, we'll make a calculator. It'll be a 4 by 3 grid:
#+-+-+-+-+
#|1|2|3|+|
#+-+-+-+-|
#|4|5|6|-|
#+-+-+-+-|
#|7|8|9|*|
#+-+-+-+-|
#and we'll print the stack after every button press. 

class CalculatorFrame(Frame): #Object-oriented style because this'll be complex.
    def __init__(self, parent):
        Frame.__init__(self, parent)
        self._stack = [] #Initialize to the empty stack.
        #Now, add each button. addButton takes the number and the coordinates
        #of the button, so here's a list of each button and its coordinates.
        buttons = [ (1, 0, 0), (2, 0, 1), (3, 0, 2), ("+", 0, 3),
                    (4, 1, 0), (5, 1, 1), (6, 1, 2), ("-", 1, 3),
                    (7, 2, 0), (8, 2, 1), (9, 2, 2), ("*", 2, 3)]
        for button in buttons:
            self._addButton(button[0], button[1], button[2])
    def _addButton(self, op, rowCoord, colCoord):
        def makeChange():
            self._stack = stackOperation(op, self._stack)
            print(self._stack)
        #Remember, command must be a function taking no arguments.
        #So, while it's tempting to say "command=makeChange()", this
        #is wrong because the value of makeChange() is void (it doesn't
        #return anything). The button will execute the command you provide
        #when mainloop tells it to do so. We could, equivalently, say
        #"command=lambda:makeChange()" which would also work. 
        button = Button(self, text=str(op), command=makeChange)
        button.grid(row=rowCoord, column=colCoord)

def calculator():
    window = Tk()
    calc = CalculatorFrame(window)
    calc.pack()
    window.mainloop()

If the way we're hooking functions onto widgets isn't clear now, work on it until it makes sense. It's going to get harder when we also have variables attached to the widgets.

> *===== ESOTERIC ASIDE =====*

> Note that earlier, with the *timesClicked* example, *timesClicked* couldn't be assigned directly. It was invalid to say

In [ ]:
#   x = 0
#   def augment():
#       x=x+1
#   Button(command=augment)

> because *x* Python would complain about a variable being referenced before assignment. We had to do

In [ ]:
#   x = [0]
#   def augment():
#       x[0] = x[0] + 1
#   Button(command=augment)

> because we're not reassigning *x* itself, we're changing the data contained in *x*.

> It would also have been invalid to say

In [ ]:
#   def augment():
#       x = [x[0] + 1]

> because that would have reassigned *x* as well.

> Here, we are getting away with

In [ ]:
#   def makeChange():
#       self._stack=stackOperation(op, self._stack)

> Which seems like it should have a problem, right? 

> We're reassigning *self._stack* directly, not just changing its data. While true, there's a subtlety that lets this work: we don't reassign *self*.

> We change the data contained in *self* (a datum called *_stack* is reassigned), but the object called *self*, which is closed over by the function, is not reassigned.

> *===== END ASIDE =====*

Okay, so that is well and good. We can hook functions onto widgets and have the GUI respond in a meaningful way to user input.

Cool!

But sometimes, the interaction with a widget is not as simple as clicking. Consider a text box: the user can enter any text they please, and our program needs a way to capture this data. The solution provided by Tkinter is inelegant, to put it mildly. This is a result of the fact that Tkinter is not really running Python, it's running Tcl. And an object in Tcl can't be easily converted to an object in Python.

The system used in Tkinter is to create a special object that is capable of performing these conversions and attach that object to the widget. 

In [ ]:
from tkinter import Checkbutton, IntVar
def checkBox():
    window = Tk()
    cbState = IntVar()
    #buttonState will be updated whenever the check button is clicked.
    def showState():
        print("The state is {0:d}".format(cbState.get()))
    cb = Checkbutton(window, text="Click me",variable=cbState, command=showState)
    #These variables have two important methods: get() and set(val).
    #Importantly, when you call set(), the widget will be updated on-screen
    #to reflect the change. 
    cb.pack()
    window.mainloop()

 Let's put all of these things together and write a program that calculates Hamming distances for two sequences. Here's the plan:

In [ ]:
#+-----------+------+
#|ENTRY BOX 1|STATUS|
#+-----------+------|
#|ENTRY BOX 2|GO BTN|
#+-----------+------+
#|S E Q U E N C E 1 |
#+------------------+
#|M I S M A T C H   |
#+------------------+
#|S E Q U E N C E 2 |
#+------------------+


Where the bottom three rows show the two sequences and have a | in themismatch row wherever they differ. 

In [ ]:
from tkinter import StringVar, Entry, Label
class HammingFrame(Frame):
    """
        Creates a frame that lets the user calculate the Hamming distance
        between two strings.
        Treat as a Frame. 
    """
    def __init__(self, parent):
        Frame.__init__(self, parent)
        #Make two string variables that will contain the user's sequences.
        self._seq1Var = StringVar()
        self._seq2Var = StringVar()
        #Entry boxes for the user to type things. These will update the
        #corresponding variables. Width is the width of the widget on-screen
        #measured in characters. It doesn't limit the number of characters
        #the user can type.
        self._seq1Entry = Entry(self, textvariable=self._seq1Var, width=50)
        self._seq2Entry = Entry(self, textvariable=self._seq2Var, width=50)
        self._seq1Entry.grid(row=0, column=0)
        self._seq2Entry.grid(row=1, column=0)

        #Often, the status will be displayed at the bottom of the window.
        #I have it to the side for no good reason other than there was nothing
        #better to put there.
        self._statusVar = StringVar()
        self._statusLabel = Label(self, textvariable=self._statusVar, width=20)
        self._statusVar.set("Waiting for input")
        self._statusLabel.grid(row=0, column=1)
        
        self._startButton = Button(self, text="Calculate", command=self._calcDist)
        self._startButton.grid(row=1, column=1)

        self._resultFrame = ResultFrame(self)
        self._resultFrame.grid(row=2, column=0, columnspan=2)


    def _calcDist(self):
        #alert the user that the program is running.
        #If the calculation was more CPU-intensive, it might be nice to provide
        #a percent-complete message or something. 
        self._statusVar.set("Calculating")
        seq1 = self._seq1Var.get()
        seq2 = self._seq2Var.get()
        if(len(seq1) != len(seq2)):
            #Hamming distance is only defined on strings of the same length.
            self._statusVar.set("Length mismatch!")
            return
        #Well, we know the lengths are the same, so show the distance.
        dist = 0
        for i in range(len(seq1)):
            if(seq1[i] != seq2[i]):
                dist += 1
        
        self._statusVar.set("dist={0:d}".format(dist))
        #Get rid of earlier output...
        self._resultFrame.clearPrevious()
        #and display the new result.
        self._resultFrame.showDistance(seq1, seq2)

class ResultFrame(Frame):
    """
        Creates a frame to show the mismatches between two strings. Starts
        empty, constructed just like a Frame.
    """
    def __init__(self, parent):
        Frame.__init__(self,parent)
        self._labels = []
        #Do nothing right now, we'll add the letters and such later.

    def clearPrevious(self):
        """
            Removes any previous content from the frame. If the frame is
            already empty, calling this does nothing. 
        """
        for label in self._labels:
            label.destroy() #destroy erases the widget from the screen.
        self._labels = [] #clear the container, otherwise we'd re-destroy
        #every label including those previously destroyed. 

    def showDistance(self, seq1, seq2):
        """
            Given two strings, shows each string and indicates the positions
            they mismatch. 
        """
        #Go through the sequences and add labels for each one. If there's
        #a mismatch, then identify it with a | character.
        for i in range(len(seq1)):
            #If I didn't need to refer to the labels in clearPrevious, I would
            #have just done "Label(self, text=seq1[i]).grid(row=0,column=i)"
            topLabel = Label(self, text=seq1[i])
            self._labels.append(topLabel)
            topLabel.grid(row=0, column=i)
            if(seq1[i] != seq2[i]): #indicate the mismatch
                midLabel=Label(self,text="|")
                self._labels.append(midLabel)
                midLabel.grid(row=1, column=i)
            botLabel = Label(self, text=seq2[i])
            self._labels.append(botLabel)
            botLabel.grid(row=2, column=i)
        
def hammingFrame():
    window = Tk()
    frame = HammingFrame(window)
    frame.pack()
    window.mainloop()

####   *Exercises*

1 - Extend the calculator developed above so that it can use multi-digit numbers. Add an 'enter' button that pushes a number on the stack, and when the user presses number buttons, combine the presses to form a single number. 

(If this is confusing, just think of entering a multi-digit number on a spreadsheet - you press each digit then press enter.)

You should probably also add a zero button for numbers that contain zeroes.

If you have the number buttons produce strings, you can just concatenate the typed numbers and convert that to a number with int().

2 - Managing files with Tkinter is different in python 3.x (the version used here) and python 2.x, which is unfortunately what many tutorials on the net reference. It's also totally different in tcl. 

The file dialogs can be imported from the *tkinter.filedialog* module, like this:

In [ ]:
from tkinter.filedialog import askopenfile

Find the commands (online) that this module makes available. (*askopenfile* is one of them, what are the others?)

Write a program that lets the user choose a file, then analyzes the chosen file to find the most common characters used.

Display the characters encounteredin order of frequency in the window.

3 - Write a program that lets the user select a .fasta file containing a peptide sequence. Let the user choose one of five proteases using radio button widgets.

On the screen, display where the provided sequence will be cleaved by the selected protease.

(It will be helpful to use a regular expression to identify the cleavage sites.)

The choice of which proteases to use is left to you.


4 - To the program written in 2, add a feature (by which I mean button) that lets the user save the sequences of the fragments in a new .fasta file.

Warn the user if they attempt to overwrite the original file.

*Uh, that's all I've got*. This is the end of Programming for Biochemists.

**You made it!**

_=_=_=_=_=_=_=_=_=_=_ _^_^_^_^_^_^_^_^_^_^_
"""CONGRATULATIONS"""
_^_^_^_^_^_^_^_^_^_^_ _=_=_=_=_=_=_=_=_=_=_